Using torch for deep learning. 

In [43]:
#standard modukles
import numpy as np
import random as random
import time
import pandas as pd
#using simpleguitk for display, is not needed for computer game
#likely not needed un this notebook 
import simpleguitk as simplegui
#for plotting 
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
#for efficient saving and loading
import pickle
#skyjp game classes and functions
from skyjo_functions4 import *
#own functions for machine learning
from ml_functions2 import *
#for machine learning
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
#time logging
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
#again tensorflow problem work before with conda install cudatoolkit, is not needed after each restart? 

In [6]:

columns=['open_pile_card']
for i in range(12):
    columns.append('own_cards_'+str(i))
for i in range(12):
    columns.append('other_player_cards_'+str(i))    
columns.append('action_take_open')
columns.append('action_discard')
columns.append('discard_value')
columns.append('id_player_card')
columns.append('numeric_player_card')            
columns.append('score_self')
columns.append('score_other')
columns.append('round')
print(columns)
print(len(columns))
print(columns[0:30])
print("last three are scores (target is differential) and round")
print(columns[30:33])

['open_pile_card', 'own_cards_0', 'own_cards_1', 'own_cards_2', 'own_cards_3', 'own_cards_4', 'own_cards_5', 'own_cards_6', 'own_cards_7', 'own_cards_8', 'own_cards_9', 'own_cards_10', 'own_cards_11', 'other_player_cards_0', 'other_player_cards_1', 'other_player_cards_2', 'other_player_cards_3', 'other_player_cards_4', 'other_player_cards_5', 'other_player_cards_6', 'other_player_cards_7', 'other_player_cards_8', 'other_player_cards_9', 'other_player_cards_10', 'other_player_cards_11', 'action_take_open', 'action_discard', 'discard_value', 'id_player_card', 'numeric_player_card', 'score_self', 'score_other', 'round']
33
['open_pile_card', 'own_cards_0', 'own_cards_1', 'own_cards_2', 'own_cards_3', 'own_cards_4', 'own_cards_5', 'own_cards_6', 'own_cards_7', 'own_cards_8', 'own_cards_9', 'own_cards_10', 'own_cards_11', 'other_player_cards_0', 'other_player_cards_1', 'other_player_cards_2', 'other_player_cards_3', 'other_player_cards_4', 'other_player_cards_5', 'other_player_cards_6', 'ot

Running it on the original random data set, of 3 million rows, using all available columns. 

In [7]:
df1=pd.read_pickle('level_rand_v2_0_600000_0.pkl')
df2=pd.read_pickle('level_rand_v2_0_600000_1.pkl')
df3=pd.read_pickle('level_rand_v2_0_600000_2.pkl')
df4=pd.read_pickle('level_rand_v2_0_600000_3.pkl')
df5=pd.read_pickle('level_rand_v2_0_600000_4.pkl')
df=pd.concat([df1,df2,df3,df4,df5],join='outer', ignore_index=True)
print(df1.shape,df.shape)
#free space
df1=0
df2=0
df3=0
df4=0
df5=0

(600000, 33) (3000000, 33)


Dividing into train and test.

In [23]:
feature_train,feature_test,feature_valid, target_train,target_test,target_valid=split_test_valid(df.iloc[:,0:30],df.score_self-df.score_other,0.5,0.8)
print(feature_train.shape,feature_test.shape,target_test.shape,feature_train.head())

1500000
2400000
(1500000, 30) (900000, 30) (900000,)    open_pile_card  own_cards_0  own_cards_1  own_cards_2  own_cards_3  \
0              -2           20            7           20           20   
1               0           -2            1            1           11   
2              -1           20           20           20           20   
3              11           11           -2            0           20   
4               4           -1            0            3            0   

   own_cards_4  own_cards_5  own_cards_6  own_cards_7  own_cards_8  ...  \
0            5           20            2           20            8  ...   
1            2           20           -1            6            7  ...   
2           20           20           20            0           20  ...   
3           20            8           -1           20           12  ...   
4            5           20           20            4            0  ...   

   other_player_cards_7  other_player_cards_8  other_play

Best xgboost for comparison. 

In [9]:
resd=np.loadtxt('xgb_v2_maxd5.txt')
rese=np.loadtxt('xgb_v2_maxd6.txt')
resf=np.loadtxt('xgb_v2_maxd7.txt')
resg=np.loadtxt('xgb_v2_maxd8.txt')
resh=np.loadtxt('xgb_v2_maxd9.txt')
resi=np.loadtxt('xgb_v2_maxd10.txt')
print(f"minimum standard deviation of (prediction-test data) {round(min(resd[3]),4)} of max-depth=5")
print(f"minimum standard deviation of (prediction-test data) {round(min(rese[3]),4)} of max-depth=6")
print(f"minimum standard deviation of (prediction-test data) {round(min(resf[3]),4)} of max-depth=7")
print(f"minimum standard deviation of (prediction-test data) {round(min(resg[3]),4)} of max-depth=8")
print(f"minimum standard deviation of (prediction-test data) {round(min(resh[3]),4)} of max-depth=9")
print(f"minimum standard deviation of (prediction-test data) {round(min(resi[3]),4)} of max-depth=10")

minimum standard deviation of (prediction-test data) 57.6014 of max-depth=5
minimum standard deviation of (prediction-test data) 57.5013 of max-depth=6
minimum standard deviation of (prediction-test data) 57.4394 of max-depth=7
minimum standard deviation of (prediction-test data) 57.4389 of max-depth=8
minimum standard deviation of (prediction-test data) 57.4242 of max-depth=9
minimum standard deviation of (prediction-test data) 57.4588 of max-depth=10


Best standard deviation is 57.4242.

In [18]:
class RegressionDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [26]:
#feature_train,feature_test,feature_valid, target_train,target_test,target_valid
feature_train, target_train = np.array(feature_train), np.array(target_train)
feature_test, target_test = np.array(feature_test), np.array(target_test)
feature_valid, target_valid = np.array(feature_valid), np.array(target_valid)

In [60]:
#use int causes error later 
train_dataset = RegressionDataset(torch.from_numpy(feature_train).float(), torch.from_numpy(target_train).float())
test_dataset = RegressionDataset(torch.from_numpy(feature_test).float(), torch.from_numpy(target_test).float())
valid_dataset = RegressionDataset(torch.from_numpy(feature_valid).float(), torch.from_numpy(target_valid).float())

In [89]:
EPOCHS = 150
BATCH_SIZE = 1024
LEARNING_RATE = 0.001
NUM_FEATURES = 30

In [90]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=valid_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

I am starting with a dense network of just two layers. 

In [91]:
class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 10)
        self.layer_out = nn.Linear(10, 1)
        
        self.relu = nn.ReLU()
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.layer_out(x)
        return (x)
    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)
        return (x)



In [92]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
###################### OUTPUT ######################cuda:0

cpu


In [93]:
model = MultipleRegression(NUM_FEATURES)
model.to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

MultipleRegression(
  (layer_1): Linear(in_features=30, out_features=10, bias=True)
  (layer_out): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)


In [87]:
loss_stats = {
    'train': [],
    "val": []
}


In [94]:
#slow with 64 batches , likely larger better as for tensorflow
#10000 at 11:36 started 11:56 epoch 12 mse 4019 4023
#100000 at 11:57 started  maybe faster but improvement is slower 12:12 epoch 10 4107 and 4108
#1024 at 12:15 13:15 validation gets worse 
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))
            
            val_epoch_loss += val_loss.item()
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

Begin training.


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 4105.55947 | Val Loss: 4083.99538
Epoch 002: | Train Loss: 4062.55707 | Val Loss: 4046.96144
Epoch 003: | Train Loss: 4036.13904 | Val Loss: 4030.64457
Epoch 004: | Train Loss: 4022.85640 | Val Loss: 4018.98036
Epoch 005: | Train Loss: 4011.00434 | Val Loss: 4008.88819
Epoch 006: | Train Loss: 4001.88734 | Val Loss: 3999.47147
Epoch 007: | Train Loss: 3995.42135 | Val Loss: 3994.25424
Epoch 008: | Train Loss: 3989.06651 | Val Loss: 3985.72731
Epoch 009: | Train Loss: 3980.45235 | Val Loss: 3982.41948
Epoch 010: | Train Loss: 3972.78714 | Val Loss: 3968.35652
Epoch 011: | Train Loss: 3957.89985 | Val Loss: 3953.70244
Epoch 012: | Train Loss: 3947.10417 | Val Loss: 3944.42325
Epoch 013: | Train Loss: 3941.40366 | Val Loss: 3940.15116
Epoch 014: | Train Loss: 3936.98633 | Val Loss: 3936.35596
Epoch 015: | Train Loss: 3933.76444 | Val Loss: 3932.97289
Epoch 016: | Train Loss: 3930.44774 | Val Loss: 3929.28759
Epoch 017: | Train Loss: 3928.12622 | Val Loss: 3928.046

KeyboardInterrupt: 

Is slower thann tensorflow. Likely not optimal use of it. 

The measure is the squered standard deviation, that means the best xgboost had an MSE of 3297.54

In [ ]:

y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
        #y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
mse = mean_squared_error(y_test, y_pred_list)
r_square = r2_score(y_test, y_pred_list)

print("Mean Squared Error :",mse)
print("R^2 :",r_square)

In [30]:
model_dl2.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

In [69]:
#a little but not much slower
model_dl2.fit(
    feature_train,target_train,
    epochs=400,batch_size=10000
)

Epoch 1/400
81/81 [==============================] - 1s 5ms/step - loss: 487.1365
Epoch 2/400
81/81 [==============================] - 0s 5ms/step - loss: 486.5871
Epoch 3/400
81/81 [==============================] - 0s 5ms/step - loss: 486.7194
Epoch 4/400
81/81 [==============================] - 0s 5ms/step - loss: 486.7383
Epoch 5/400
81/81 [==============================] - 0s 5ms/step - loss: 486.6437
Epoch 6/400
81/81 [==============================] - 0s 5ms/step - loss: 486.5973
Epoch 7/400
81/81 [==============================] - 0s 5ms/step - loss: 486.6439
Epoch 8/400
81/81 [==============================] - 0s 5ms/step - loss: 486.5850
Epoch 9/400
81/81 [==============================] - 0s 5ms/step - loss: 486.6941
Epoch 10/400
81/81 [==============================] - 0s 5ms/step - loss: 486.6545
Epoch 11/400
81/81 [==============================] - 0s 5ms/step - loss: 486.6057
Epoch 12/400
81/81 [==============================] - 0s 5ms/step - loss: 486.7681
Epoch 13/400


81/81 [==============================] - 0s 5ms/step - loss: 486.5155
Epoch 197/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3952
Epoch 198/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4389
Epoch 199/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4365
Epoch 200/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4265
Epoch 201/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3818
Epoch 202/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4822
Epoch 203/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4144
Epoch 204/400
81/81 [==============================] - 0s 5ms/step - loss: 486.5041
Epoch 205/400
81/81 [==============================] - 0s 5ms/step - loss: 486.5059
Epoch 206/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4005
Epoch 207/400
81/81 [==============================] - 0s 5ms/step - loss: 486.4969
Epoch 

81/81 [==============================] - 0s 5ms/step - loss: 486.1865
Epoch 391/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3273
Epoch 392/400
81/81 [==============================] - 0s 5ms/step - loss: 486.2701
Epoch 393/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3063
Epoch 394/400
81/81 [==============================] - 0s 6ms/step - loss: 486.2314
Epoch 395/400
81/81 [==============================] - 1s 6ms/step - loss: 486.2650
Epoch 396/400
81/81 [==============================] - 0s 6ms/step - loss: 486.3355
Epoch 397/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3589
Epoch 398/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3851
Epoch 399/400
81/81 [==============================] - 0s 5ms/step - loss: 486.3185
Epoch 400/400
81/81 [==============================] - 0s 6ms/step - loss: 486.3044


In [70]:
mtr=np.shape(feature_train)
mte=np.shape(feature_test)
#takes a minute thus doing too often taht would slow it down a lot 
prediction_train2 = model_dl2.predict(feature_train[0:mtr[0]].reshape(mtr[0],mtr[1]))
prediction_test2 = model_dl2.predict(feature_test[0:mte[0]].reshape(mte[0],mte[1]))

15156/15156 [==============================] - 18s 1ms/step


In [71]:
print(f"standard deviation of test {np.round(np.std(target_test),4)} points")
print(f"standard deviation of train {np.round(np.std(target_train),4)} points")
print(f"standard deviation of test {np.round(np.std(target_test-prediction_test2.T),4)} points")
print(f"standard deviation of train {np.round(np.std(target_train-prediction_train2.T),4)} points")

standard deviation of test 24.0924 points
standard deviation of train 24.1769 points
standard deviation of test 22.1872 points
standard deviation of train 22.0453 points


After 20 22.8258 and 22.6636. After 220 22.3098 and  22.1292. Clearly better improving now than with 1 relu layer.  Still after 200 it seems to stall. After 400 more I get 22.2016 and  22.0549.
After 400 more I get 22.1872 and  22.0453. seems to stall. 

Compare to xgb on the data which has 21.7426  and 20.2503 for using the same data and when  21.5263 and 21.3506  the feature engetring is used. 

I  also use the feature engering data. 

Or adding even more layers since there no clear overfitting. 

In [75]:
#even more layers
model_dl3 = Sequential(
    [              
        tf.keras.Input(shape=(30,)),   
        Dense(20, activation="relu", name="layer1"),
        Dense(12, activation="relu", name="layer2"),     
        Dense(4, activation="relu", name="layer3"),        
        Dense(1, activation="linear", name="layer4"),
    ], name = "model3"
) 
model_dl3.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)


In [77]:
#110
model_dl3.fit(
    feature_train,target_train,
    epochs=100,batch_size=10000
)
#is improving fast 492, but that is still worse than what the 2 layer model had before 
#more running needed

Epoch 1/100
81/81 [==============================] - 1s 7ms/step - loss: 521.1448
Epoch 2/100
81/81 [==============================] - 0s 6ms/step - loss: 520.0187
Epoch 3/100
81/81 [==============================] - 0s 6ms/step - loss: 518.9448
Epoch 4/100
81/81 [==============================] - 0s 6ms/step - loss: 517.9707
Epoch 5/100
81/81 [==============================] - 0s 6ms/step - loss: 517.0075
Epoch 6/100
81/81 [==============================] - 1s 6ms/step - loss: 515.9370
Epoch 7/100
81/81 [==============================] - 1s 6ms/step - loss: 515.0507
Epoch 8/100
81/81 [==============================] - 0s 6ms/step - loss: 513.9315
Epoch 9/100
81/81 [==============================] - 0s 6ms/step - loss: 513.2241
Epoch 10/100
81/81 [==============================] - 0s 6ms/step - loss: 512.4639
Epoch 11/100
81/81 [==============================] - 0s 6ms/step - loss: 511.8160
Epoch 12/100
81/81 [==============================] - 0s 6ms/step - loss: 511.4802
Epoch 13/100
